In [ ]:
%load_ext autoreload
%autoreload 2

from data_prep import DataPreparation, filtering_data
from lstm import LSTMModelTrainerAttention
from rf import RandomForestTrainer
from lgbm import LightGBMTrainer
from catboost_trainer import CatBoostTrainer
import os
import dill as pickle
import numpy as np
import pandas as pd
from analysis import plot_correlation_heatmap, get_ETO_DEP
from lstm import LSTMModelTrainerAttention, calculate_and_plot_errors#, LSTMModelTrainer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.ticker as ticker
from tqdm import tqdm
from lstm import LSTMRollingForecaster
from collections import defaultdict
import matplotlib.pyplot as plt
from helper import extract_flight_details
from lstm import LSTMModelTrainerAttention, LSTMRollingForecaster
from rf import RandomForestTrainer
from lgbm import LightGBMTrainer
from catboost_trainer import CatBoostTrainer
from transformer import TransformerTrainer
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
output= r'C:\Users\iLabs_6\Documents\Tex\allwithCBAS'
modelname = 'extended_df_ATOT'

 
with  open(os.path.join(output, modelname), 'rb') as f:  
    extended_df = pickle.load(f)

extended_df['cap_DEP'].fillna(extended_df['cap_DEP'].mean(), inplace=True)
extended_df['cap_DES'].fillna(extended_df['cap_DES'].mean(), inplace=True)
extended_df = extended_df.loc[:, ~extended_df.columns.str.contains('t_to_eobt', case=False)]
extended_df = extended_df.loc[:, ~extended_df.columns.str.contains('t_to_atot', case=False)]
extended_df = extended_df.drop_duplicates()

pd.set_option("display.max_columns", None)
X, y, colnames = filtering_data(extended_df, airport ='EHAM', save=True)
X = X.drop(['ADES_EHAM', 'ADESLong', 'ADESLat', 'cbasentry'], axis =1)
X.describe()

# output= r'C:\Users\iLabs_6\Documents\Tex\realtimetest3'
modelname = 'extended_df_ETOT'


with  open(os.path.join(output, modelname), 'rb') as f:  
    extended_real= pickle.load(f)
extended_real = extended_real.loc[:, ~extended_real.columns.str.contains('t_to_eobt', case=False)]
extended_real = extended_real.loc[:, ~extended_real.columns.str.contains('t_to_atot', case=False)]
extended_real = extended_real.drop_duplicates()

pd.set_option("display.max_columns", None)
print(f'{extended_real}')
X_r, y_r, colnames = filtering_data(extended_real, airport ='EHAM', save=False)
X_r = X_r.drop(['ADES_EHAM', 'ADESLong', 'ADESLat', 'cbasentry'], axis =1)
# X.fillna(0, inplace=True)
print(f'xxxxx = {X_r}')

data_prep = DataPreparation()


X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, time_horizons, cbaslabels = data_prep.fit_transform_data(X, y)
X_real, ETOT_horizons, cbaslabels = data_prep.transform_data(X_r)



In [ ]:

# Save to a pickle file
output_file = r"C:\Users\iLabs_6\Documents\Tex\AirTrafficDelays\output\error_analysis_data_all.pkl"

# print(f"Data saved to {output_file}")

with open(output_file, 'rb') as file:
    loaded_data = pickle.load(file)

# Access the variables
abs_error_dict = loaded_data['abs_error_dict']
error_dict = loaded_data['error_dict']
e_adep = loaded_data['e_adep']
e_dist = loaded_data['e_dist']
e_wd = loaded_data['e_wd']
e_time = loaded_data['e_time']
adeplonglist = loaded_data['adeplonglist']
adeplatlist = loaded_data['adeplatlist']
adeplatlonlist = loaded_data['adeplatlonlist']
bucketed_abs_errors = loaded_data['bucketed_abs_errors']
bucketed_errors = loaded_data['bucketed_errors']
mean_abs_errors_by_bucket = loaded_data['mean_abs_errors_by_bucket']
mean_errors_by_bucket = loaded_data['mean_errors_by_bucket']
std_errors_by_bucket = loaded_data['std_errors_by_bucket']
sorted_abs_buckets = loaded_data['sorted_abs_buckets']
sorted_buckets = loaded_data['sorted_buckets']
sorted_std_buckets = loaded_data['sorted_std_buckets']
abs_times = loaded_data['abs_times']
abs_errors = loaded_data['abs_errors']
times = loaded_data['times']
errors = loaded_data['errors']
std_errors = loaded_data['std_errors']

print("Data successfully loaded from", output_file)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
from collections import defaultdict
target_length=61
warnings.filterwarnings("ignore", category=UserWarning)

# Function to plot a dictionary of errors
def plot_dict_errors(error_dict, title):
    plt.figure(figsize=(12, 6))
    maelist = []
    for key, errors in error_dict.items():
        # print(f'{key=}')
        # print(f'{len(errors)=}')
        mae_per_timestep = np.nanmean(errors, axis=0)
        maelist.append(mae_per_timestep)
        plt.plot(ETOT_horizons[:target_length], mae_per_timestep, marker='o', label=key)
    plt.plot(ETOT_horizons[:target_length], np.nanmean(maelist,axis=0), marker='', label='Mean', color='red')

    plt.xlabel('Timestep')
    plt.xticks(ETOT_horizons[:target_length], rotation='vertical')
    plt.ylabel('Mean Absolute Error (MAE)')
    plt.title(title)
    plt.legend(title="Keys")
    plt.tight_layout()
    plt.grid(True)
    plt.show()

# plot_dict_errors(e_adep, "MAE per Timestep - ADEP")
# plot_dict_errors(e_dist, "MAE per Timestep - Distance")
plot_dict_errors(e_wd, "MAE per Timestep - Day")
# plot_dict_errors(adeplonglist, "MAE per Timestep - Long")
# plot_dict_errors(adeplatlist, "MAE per Timestep - Lat")

bucketed_errors = defaultdict(list)

for time_key, errors in e_time.items():
    time_to_cbas = time_key  # Use time_key as time to CBAS (assuming time_key is in minutes)
    bucket = int(time_to_cbas//180 )  *3 # Group into 60 minute intervals
    bucketed_errors[bucket].append(errors)

bucketed_dist = defaultdict(list)
for time_key, errors in e_dist.items():
  # Use time_key as time to CBAS (assuming time_key is in minutes)
    bucket = int(time_key//5000 )*5000   # Group into 60 minute intervals
    bucketed_dist[bucket].append(errors)
# print(f'{bucketed_dist.keys()}')

# Plot the bucketed errors for e_time
plt.figure(figsize=(12, 6))
for bucket, edict in bucketed_errors.items():
    # print(f'{bucket=}')
    # print(f'{edict=}')
    mae_per_timestep = np.nanmean(edict[0], axis=0)

    plt.plot(ETOT_horizons[:target_length], mae_per_timestep, marker='o', label=f"{bucket} hour bucket")
plt.xlabel('Timestep')
plt.xticks(ETOT_horizons[:target_length], rotation='vertical')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('MAE for Each Timestep - Time Buckets')
plt.legend(title="Time Buckets")
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
# Create a DataFrame for the plot
data = pd.DataFrame({'Airport': [x.split('ADEP_')[1] for x in e_adep.keys()], 'MAE': [np.nanmean(x) for x in e_adep.values()]})


# Number of airports
N = len(data)

# Compute angle for each bar
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Complete the loop

# Heights of bars
heights = data['MAE'].tolist()
heights += heights[:1]  # Complete the loop

# Initialize the plot
fig = plt.figure(figsize=(8, 8))
ax = plt.subplot(111, polar=True)

# Remove labels and ticks
ax.set_xticks([])
ax.set_yticks([])

# Set the minimum radius (distance from center where bars start)
bottom = 2.0  # Adjust this value to change how far bars start from the center

# Add bars
bar_width = 2 * np.pi / N * 0.8  # Adjust the 0.8 factor to change bar width (spacing between bars)
bars = ax.bar(angles[:-1], heights[:-1], width=bar_width, bottom=bottom, color='salmon', alpha=0.6)

# Adjust the maximum radius to ensure all bars and labels fit
max_height = bottom + max(heights) + max(heights) * 0.1  # Add some padding
ax.set_ylim(0, max_height)

# Add labels
for i, bar in enumerate(bars):
    angle = angles[i]
    rotation = np.degrees(angle)
    alignment = ''
    if rotation >= 270 or rotation <= 90:
        alignment = 'left'
        rotation = rotation
    else:
        alignment = 'right'
        rotation = rotation + 180
    # Position the label at the top of the bar
    ax.text(
        angle,
        bottom + heights[i] + max(heights) * 0.05,  # Adjust position based on 'bottom' and 'heights'
        data['Airport'].iloc[i],
        ha=alignment,
        va='center',
        rotation=rotation,
        rotation_mode='anchor',
        fontsize=8
    )

plt.title('Circular Barplot of MAE per Airport', y=1.08)
plt.show()

In [ ]:

airport_errors = [[e for e in e_adep[airport][0] if not np.isnan(e)] for airport in e_adep.keys()]
airport_names = [x.split('ADEP_')[1] for x in e_adep.keys()]
print(f'{airport_errors=}')
# Create boxplots
plt.figure(figsize=(15, 6))
plt.boxplot(airport_errors, labels=airport_names, showfliers=False)
plt.xticks(rotation=45, ha='right')
plt.xlabel('Airport')
plt.ylabel('Error (minutes)')
plt.title('Boxplots of Errors per Airport')
plt.tight_layout()
plt.show()
airport_errors[22]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm
tempmae = {}
print(f'{bucketed_errors.keys()}')
amounts = {a: len(l) for a,l in e_adep.items() }
sorted_amounts = {k: v for k, v in sorted(amounts.items(), key=lambda item: item[1], reverse=True)}

topairports = list(sorted_amounts.keys())[:25]
print(f'{topairports=}')
te_adep = {k:v for k,v in e_adep.items() if k in topairports}
hourdata = pd.DataFrame({'Airport':  [x.split('ADEP_')[1] for x in te_adep.keys()], 'MAE': [np.nanmean(x) for x in te_adep.values()]})
print(f'{sorted_amounts=}')
# Max number of airports to display
max_airports = 25
data_grid = hourdata.iloc[:max_airports].reset_index(drop=True)
num_airports = len(data_grid)

# Compute circle sizes (adjust scaling factor as needed)
max_mae = data_grid['MAE'].max()
sizes = data_grid['MAE'] / max_mae * 3000  # Adjust the multiplier to change circle sizes

# Normalize MAE values for color mapping
norm = mcolors.Normalize(vmin=data_grid['MAE'].min(), vmax=data_grid['MAE'].max())
cmap = cm.get_cmap('coolwarm')
colors = [cmap(norm(value)) for value in data_grid['MAE']]

# Grid dimensions
grid_rows = 5
grid_cols = 5

fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(grid_cols*1.5, grid_rows*1.5))
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i < num_airports:
        mae_value = data_grid['MAE'].iloc[i]
        color = colors[i]
        ax.scatter(0.5, 0.5, s=sizes.iloc[i], color=color, alpha=0.6)
        ax.text(0.5, 0.5, f"{mae_value:.1f}", ha='center', va='center', fontsize=12)
        ax.axis('off')  # Remove the box around the circle
        ax.set_title(data_grid['Airport'].iloc[i], fontsize=10)
    else:
        ax.axis('off')  # Hide unused subplots

plt.suptitle('MAE per Airport', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
# e_time

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# Assume `e_adep` is already available with airport data
amounts = {a: len(l) for a, l in e_adep.items()}
sorted_amounts = {k: v for k, v in sorted(amounts.items(), key=lambda item: item[1], reverse=True)}

# Select the top 25 airports
topairports = list(sorted_amounts.keys())#[:25]
te_adep = {k: v for k, v in e_adep.items() if k in topairports}

# Create a DataFrame for top airports
hourdata = pd.DataFrame({
    'Airport': [x.split('ADEP_')[1] for x in te_adep.keys()],
    'MAE': [np.nanmean(x) for x in te_adep.values()],
    'Size': [sorted_amounts[k] for k in te_adep.keys()]  # Add Size based on sorted_amounts
})

# Max number of airports to display
max_airports = 25
data_grid = hourdata.iloc[:max_airports].reset_index(drop=True)
num_airports = len(data_grid)

# Compute circle sizes based on "Size" column (adjust scaling factor as needed)
max_size = data_grid['Size'].max()
sizes = data_grid['Size'] / max_size * 3000  # Adjust the multiplier to change circle sizes

# Normalize MAE values for color mapping
norm = mcolors.Normalize(vmin=data_grid['MAE'].min(), vmax=data_grid['MAE'].max())
cmap = cm.get_cmap('coolwarm')
colors = [cmap(norm(value)) for value in data_grid['MAE']]

# Grid dimensions
grid_rows = 5
grid_cols = 5

fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(grid_cols*1.5, grid_rows*1.5))
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i < num_airports:
        mae_value = data_grid['MAE'].iloc[i]
        size_value = sizes.iloc[i]*1.3
        color = colors[i]
        ax.scatter(0.5, 0.5, s=size_value, color=color, alpha=0.6)
        ax.text(0.5, 0.5, f"{mae_value:.1f}", ha='center', va='center', fontsize=12)
        ax.axis('off')  # Remove the box around the circle
        ax.set_title(data_grid['Airport'].iloc[i], fontsize=12)
    else:
        ax.axis('off')  # Hide unused subplots

plt.suptitle('MAE per Airport', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# Assume `e_adep` is already available with airport data
amounts = {a: len(l) for a, l in e_adep.items()}
sorted_amounts = {k: v for k, v in sorted(amounts.items(), key=lambda item: item[1], reverse=True)}

# Select the top 25 airports
topairports = list(sorted_amounts.keys())[:25]
te_adep = {k: v for k, v in e_adep.items() if k in topairports}

# Create a DataFrame for top airports
hourdata = pd.DataFrame({
    'Airport': [x.split('ADEP_')[1] for x in te_adep.keys()],
    'RMSE': [np.sqrt(np.nanmean(np.array(x)**2)) for x in te_adep.values()],  # Compute RMSE
    'Size': [sorted_amounts[k] for k in te_adep.keys()]  # Add Size based on sorted_amounts
})

# Max number of airports to display
max_airports = 25
data_grid = hourdata.iloc[:max_airports].reset_index(drop=True)
num_airports = len(data_grid)

# Compute circle sizes based on "Size" column (adjust scaling factor as needed)
max_size = data_grid['Size'].max()
sizes = data_grid['Size'] / max_size * 3000  # Adjust the multiplier to change circle sizes

# Normalize RMSE values for color mapping
norm = mcolors.Normalize(vmin=data_grid['RMSE'].min(), vmax=data_grid['RMSE'].max())
cmap = cm.get_cmap('coolwarm')
colors = [cmap(norm(value)) for value in data_grid['RMSE']]

# Grid dimensions
grid_rows = 5
grid_cols = 5

fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(grid_cols*1.5, grid_rows*1.5))
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i < num_airports:
        rmse_value = data_grid['RMSE'].iloc[i]
        size_value = sizes.iloc[i] * 1.3
        color = colors[i]
        ax.scatter(0.5, 0.5, s=size_value, color=color, alpha=0.6)
        ax.text(0.5, 0.5, f"{rmse_value:.1f}", ha='center', va='center', fontsize=12)
        ax.axis('off')  # Remove the box around the circle
        ax.set_title(data_grid['Airport'].iloc[i], fontsize=12)
    else:
        ax.axis('off')  # Hide unused subplots

plt.suptitle('RMSE per Airport', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.patches import Rectangle

# Assume `e_adep` is already available with airport data
amounts = {a: len(l) for a, l in e_adep.items()}
sorted_amounts = {k: v for k, v in sorted(amounts.items(), key=lambda item: item[1], reverse=True)}

# Select the top 25 airports
topairports = list(sorted_amounts.keys())[:25]
te_adep = {k: v for k, v in e_adep.items() if k in topairports}

# Create a single DataFrame for top airports
hourdata = pd.DataFrame({
    'Airport': [x.split('ADEP_')[1] for x in te_adep.keys()],
    'MAE': [np.nanmean(x) for x in te_adep.values()],
    'RMSE': [np.sqrt(np.nanmean(np.array(x)**2)) for x in te_adep.values()],
    'Size': [sorted_amounts[k] for k in te_adep.keys()]  # Add Size based on sorted_amounts
}).sort_values(by='Size', ascending=False).reset_index(drop=True)  # Ensure consistent sorting

# Max number of airports to display
max_airports = 25
data_grid = hourdata.iloc[:max_airports]
num_airports = len(data_grid)

# Define a function for the visualization
def plot_airport_metrics(metric_column, title):
    # Compute circle sizes based on "Size" column (adjust scaling factor as needed)
    max_size = data_grid['Size'].max()
    sizes = data_grid['Size'] / max_size * 5000  # Adjust the multiplier to change circle sizes

    # Normalize metric values for color mapping
    norm = mcolors.Normalize(vmin=data_grid[metric_column].min(), vmax=data_grid[metric_column].max())
    cmap = cm.get_cmap('coolwarm')
    colors = [cmap(norm(value)) for value in data_grid[metric_column]]

    # Grid dimensions
    grid_rows = 5
    grid_cols = 5

    fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(grid_cols * 2, grid_rows * 2))
    axes = axes.flatten()

    for i, ax in enumerate(axes):
        if i < num_airports:
            metric_value = data_grid[metric_column].iloc[i]
            size_value = sizes.iloc[i] * 1.3
            color = colors[i]
            ax.scatter(0.5, 0.5, s=size_value, color=color, alpha=0.6)
            ax.text(0.5, 0.5, f"{metric_value:.1f}", ha='center', va='center', fontsize=15)
            ax.axis('off')  # Remove the axes
            ax.set_title(data_grid['Airport'].iloc[i], fontsize=15)
        else:
            ax.axis('off')  # Hide unused subplots

    # Add one large rectangle around all circles
    fig.patches.extend([
        Rectangle(
            (0.02, 0.07),  # Lower-left corner of the rectangle
            0.96, 0.85,   # Width and height (relative to figure size)
            transform=fig.transFigure,  # Use figure-relative coordinates
            color="black",
            linewidth=1,
            fill=False  # Keep the rectangle outline only
        )
    ])

    plt.suptitle(title, fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# Plot MAE
plot_airport_metrics('MAE', 'MAE per Airport')

# Plot RMSE
plot_airport_metrics('RMSE', 'RMSE per Airport')


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# Assume `e_adep` is already available with airport data
amounts = {a: len(l) for a, l in e_adep.items()}
sorted_amounts = {k: v for k, v in sorted(amounts.items(), key=lambda item: item[1], reverse=True)}

# Select the top 25 airports
topairports = list(sorted_amounts.keys())[:25]
te_adep = {k: v for k, v in e_adep.items() if k in topairports}

# Create a single DataFrame for top airports
hourdata = pd.DataFrame({
    'Airport': [x.split('ADEP_')[1] for x in te_adep.keys()],
    'MAE': [np.nanmean(x) for x in te_adep.values()],
    'RMSE': [np.sqrt(np.nanmean(np.array(x)**2)) for x in te_adep.values()],
    'Size': [sorted_amounts[k] for k in te_adep.keys()]  # Add Size based on sorted_amounts
}).sort_values(by='Size', ascending=False).reset_index(drop=True)  # Ensure consistent sorting

# Function to plot using Seaborn
def plot_airport_metrics_seaborn(metric_column, title):
    data = hourdata.copy()
    data['Metric'] = data[metric_column]
    data['Size_Scaled'] = data['Size'] / data['Size'].max() * 3000  # Adjust scaling factor as needed

    # Create grid positions
    grid_rows = 5
    grid_cols = 5
    data['Grid_X'] = np.tile(np.arange(grid_cols), grid_rows)
    data['Grid_Y'] = np.repeat(np.arange(grid_rows)[::-1], grid_cols)

    # Normalize metric values for color mapping
    norm = plt.Normalize(data['Metric'].min(), data['Metric'].max())
    cmap = sns.color_palette('coolwarm', as_cmap=True)

    # Set up the figure
    plt.figure(figsize=(grid_cols * 2, grid_rows * 2))

    # Create scatter plot
    scatter = sns.scatterplot(
        data=data,
        x='Grid_X',
        y='Grid_Y',
        size='Size_Scaled',
        sizes=(100, 3000),
        hue='Metric',
        palette=cmap,
        legend=False
    )

    # Add annotations for metric values and airport codes
    for _, row in data.iterrows():
        plt.text(row['Grid_X'], row['Grid_Y'], f"{row['Metric']:.1f}",
                 horizontalalignment='center', verticalalignment='center', fontsize=12, color='white', weight='bold')
        plt.text(row['Grid_X'], row['Grid_Y'] - 0.4, row['Airport'],
                 horizontalalignment='center', verticalalignment='center', fontsize=10)

    # Remove axes for a cleaner look
    plt.axis('off')

    # Add title
    plt.suptitle(title, fontsize=16)

    # Adjust layout
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# Plot MAE
plot_airport_metrics_seaborn('MAE', 'MAE per Airport')

# Plot RMSE
plot_airport_metrics_seaborn('RMSE', 'RMSE per Airport')

